Target: 
explore data, 
build basic linear regression that explains "wage", 
interpret results, 
conduct test for heteroskedasticity

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import statsmodels.api as sm
from pandas.plotting import scatter_matrix
from statsmodels.stats.diagnostic import het_white
import seaborn as sns

In [2]:
df = pd.read_stata("Labour2.dta")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Labour2.dta'

In [ ]:
# firstly observe shape and size of the dataset
print(df.shape)
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.drop(columns="obs")

In [ ]:
# df.plot.scatter(x="capital", y="wage")
# plt.xlabel("Capital")
# plt.ylabel("Wage")
# plt.title("Wage against Capital");

In [ ]:
# mask to filter 1 row with max(value) of "capital"
print(df[df["capital"] == max(df["capital"])])


In [ ]:
# specify list of columns with outliers
columns = list(df.columns)

# create empty dictionary with columns as keys
def quar_dict(columns):  # takes as input list of column names
    quartiles = {}
    for column in columns:
        quartiles[column] = None
    return quartiles

# create empty dictionary with columns as keys by list comprehension
def quar_dict2(columns):  # takes as input list of column's names
    quartiles = {column: None for column in columns}
    return quartiles

# fill empty dictionary with percentiles
def percentiles(empty_dict, columns, percentiles=[1, 99]):
# takes empty_dictionary with column names as keys, columns as a list of column's names
# more than 2 percentiles could be specified
    for column in columns:
        a = list(np.percentile(df[column], percentiles))
        empty_dict[column] = a
    return empty_dict

# invoke functions above
percentile = percentiles(quar_dict2(columns), columns, [0, 99])
pd.DataFrame(percentile)

In [ ]:
# Subset data: Remove outliers for specified columns
def remove_out(df, dict_out):
    df1 = df
    for column in list(dict.keys(dict_out)):  # Expected that in given dict_out keys = column names
        mask_area = df1[column].between(dict_out[column][0], dict_out[column][1])  
        # Boolean series whether or not value between given percentiles (values in dict_out) fit
        df1 = df1[mask_area]  # Masks applied one by one
    print(f"Number of lost rows {len(df.index) - len(df1.index)}") # How many rows've been lost
    return df1
        
        
df1= remove_out(df, percentile)


In [ ]:
# #  trimmed scatter
# df1.plot.scatter(x="capital", y="wage")
# plt.xlabel("Capital")
# plt.ylabel("Wage")
# plt.title("Wage against Capital");

In [ ]:
# scatter_matrix(df, alpha=0.2, figsize=(6, 6), diagonal="kde");

In [ ]:
# scatter_matrix(df1, alpha=0.2, figsize=(6, 6), diagonal="kde");

In [ ]:
# log_wage = np.log(df["wage"])
# log_wage.plot.hist();


In [ ]:
#  Divide dataset into dependent vector "y" and explanatory matrix "X"
target = "wage"
y = df1[target]
# X = sm.add_constant(df1["capital"])
X = sm.add_constant(df1.drop(columns=target))

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

TESTS FOR HETEROSKEDASTICITY

In [ ]:
# Heteroskedasticity tests
# White test
residuals_array = results.resid

#perform White's test
white_test = sm.stats.diagnostic.het_white(residuals_array,  results.model.exog)

#define labels to use for output of White's test
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']

#print results of White's test
print(dict(zip(labels, np.around(white_test, 2))))



In [ ]:
# plotted residuals
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results, 'capital', fig=fig);



In [ ]:
#  trimmed scatter with regression line
sns.regplot(x="capital", y="wage", data=df1)
plt.xlabel("Capital")
plt.ylabel("Wage")
plt.title("Wage against Capital");

In [ ]:
# df1["wage"] = np.log(df["wage"])
# # df1["capital"] = np.log(df["capital"])

In [ ]:
# sns.regplot(x="capital", y="wage", data=df1)
# plt.xlabel("Capital")
# plt.ylabel("Wage")
# plt.title("Wage against Capital");